In [ ]:
!pip install torch torchvision


In [ ]:
import torch
import torchvision.models as models

# Load a ResNet50 model
model = models.resnet50()

# Modify the final fully connected layer to have 2 outputs
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)

# Load the model weights
model_path = '/content/resnet50_deepfake_detector.pth'  # Use your file path
checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)

# Set the model to evaluation mode
model.eval()


<ipython-input-2-dc1dc857ddc2>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=torch.device('cpu'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from torchvision import transforms
from PIL import Image

# Define preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the image and convert it to RGB
image_path = '/content/WIN_20240827_15_12_40_Pro.jpg'  # Replace with your image filename
image = Image.open(image_path).convert("RGB")  # Convert to RGB to handle images with alpha channel
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # Create a batch


In [ ]:
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension
output = model(input_tensor)


In [ ]:
import torch.nn.functional as F

# Apply softmax to get probabilities for both classes
probabilities = F.softmax(output, dim=1)
deepfake_score = probabilities[0, 1].item()  # Assuming index 1 is the deepfake class

print("Deepfake probability:", deepfake_score)


Deepfake probability: 0.9999629259109497


In [ ]:
threshold = 0.5
if deepfake_score > threshold:
    print("Detected as deepfake")
else:
    print("Detected as real")

Detected as deepfake


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=False)  # Set pretrained=False if you're not using ImageNet weights

# Modify the final fully connected layer for binary classification (real or fake)
model.fc = nn.Linear(model.fc.in_features, 2)

# Load the model weights into the model
model.load_state_dict(torch.load('/content/resnet50_deepfake_detector.pth'))

# Set the model to evaluation mode
model.eval()

# Function to extract frames from a video
def extract_frames(video_path, frame_rate=1):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps / frame_rate)

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % interval == 0:
            frames.append(frame)
        frame_count += 1
    cap.release()
    return frames

# Preprocessing function for frames
def preprocess_frame(frame):
    # Convert frame to RGB and resize it for the model input
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)

    # Define the required transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to the model's expected input size
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (ImageNet stats)
    ])

    frame = transform(frame)
    frame = frame.unsqueeze(0)  # Add batch dimension (for one image)
    return frame

# Function to predict on a video
# Function to predict on a video
def predict_on_video(video_path):
    frames = extract_frames(video_path)
    predictions = []

    # Process each frame
    for frame in frames:
        preprocessed_frame = preprocess_frame(frame)
        with torch.no_grad():  # No gradient calculation during inference
            output = model(preprocessed_frame)  # Get the model's prediction
            _, pred = torch.max(output, 1)  # Get the predicted class (real or fake)
            predictions.append(pred.item())

    # Convert predictions to integer type before applying np.bincount
    predictions = np.array(predictions, dtype=int)

    # Aggregate results (e.g., majority voting or simply returning the most frequent class)
    result = np.argmax(np.bincount(predictions))  # Majority voting to decide the final class

    if result == 0:  # Assuming 0 is 'real' and 1 is 'deepfake'
        print("Video is Real.")
    else:
        print("Video is a Deepfake.")


# Example usage: Test on a video file
video_path = '/content/Train window – OpenAI Sora AI Video Example.mp4'  # Replace with your actual video path
predict_on_video(video_path)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-9-a3af0bc94e04>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Video is a Deepfake.
